# Structured Data

Often, when you are using Claude to generate structured data (e.g., code, CSV, or a bulleted list of text), Claude prepends and appends commentary. For example, if you ask Claude to generate JSON for KRP's current stock price, you may get any output like the following.

```text
Based on the search results, here's the JSON for KRP's (Kimbell Royalty Partners) current stock price:
```
```json
json{
  "ticker": "KRP",
  "company_name": "Kimbell Royalty Partners, LP",
  "current_price": 11.50,
  "currency": "USD",
  "price_change": -0.29,
  "price_change_percent": -2.46,
  "previous_close": 12.28,
  "open": 12.29,
  "day_range": {
    "low": 11.36,
    "high": 11.79
  },
  "52_week_range": {
    "low": 10.98,
    "high": 16.59
  },
  "volume": 1214520,
  "average_volume": 480609,
  "market_cap": 1272000000,
  "dividend_yield": 13.57,
  "pe_ratio": null,
  "eps": -0.09,
  "as_of_date": "2026-01-03",
  "exchange": "NYSE"
}
```
```text
The stock closed at $11.50, down $0.29 or 2.46% CNN from the previous close.
```

There may be situations where you'd like Claude to only produce the JSON (and omit the commentary). To encourage Claude to only provide the JSON, we can use **prefilled assistant messages** in combination with **stop sequences** (as initially discussed in `005_controlling_output.ipynb`).

**Note:** The example above was copied from a real response from [claude.ai](https://claude.ai/). In this response, the price data is wrong, which is a good reminder of the limitations of contemporary LLMs. For this coding example, however, the accuracy of the information is not relevant.

## Set up the Environment and Create Helper Functions

**Note:** For this example, which relies on asking Claude to do a bit more research, we are using `claude-sonnet-4-5`, which is more powerful than the `claude-sonnet-4-0` model we've used in other examples.

**Note:** The `chat` function accepts stop sequences.

In [1]:
# Load environment variables and create client

from dotenv import load_dotenv
from anthropic import Anthropic

load_dotenv()   # Load environment variables from .env file

# Create an API client
client = Anthropic()
model = "claude-sonnet-4-5"

In [2]:
# Define helper functions for managing message history and chatting with Claude

def add_user_message(messages : list[str], content : str):
    """Append a user message to a message (aka conversation) history.

    Args:
        messages (list[str]): The message history.
        content (str): A user message to append to the message history.
    """
    user_message = { "role": "user", "content": content }
    messages.append(user_message)

def add_assistant_message(messages : list[str], content : str):
    """Append an assistant message to a message (aka conversation) history.

    Args:
        messages (list[str]): The message history.
        content (str): An assistant message to append to the message history.
    """
    assistant_message = { "role": "assistant", "content": content }
    messages.append(assistant_message)

def chat(messages : list[str], stop_sequences: list[str] = []) -> str:
    """Chat with Claude.

    Args:
        messages (list[str]): The message (aka conversation) history.
        stop_sequences (list[str], optional): Stop sequences for Claude's response. Defaults to [].

    Returns:
        str: Claude's text response.
    """
    response = client.messages.create(
        model=model,
        max_tokens=1000,
        messages=messages,
        system="Provide concise answers.",
        stop_sequences=stop_sequences
    )
    return response.content[0].text

## Chat with Claude

In [3]:
import json     # For pretty-printing JSON output

messages = []

# Make is so Claude is biased towards Python as the best programming language for beginners
add_user_message(messages, "Generate JSON for KRP's current stock price.")
add_assistant_message(messages, "```json")

response = chat(messages, stop_sequences=["```"])
json.loads(response)


{'symbol': 'KRP',
 'company_name': 'Kimbell Royalty Partners, LP',
 'current_price': None,
 'currency': 'USD',
 'last_updated': None,
 'note': "I don't have access to real-time stock market data. Please use a financial data service like Yahoo Finance, Bloomberg, Google Finance, or your brokerage platform to get the current stock price for KRP."}

## Exercise
In this exercise, you will use message prefilling and stop sequences *only* to get three different commands in a single response. There shouldn't be any comments or explanation. You will use the exact prompt listed below.

**Hint:** message prefilling isn't limited to just characters like ```

### Baseline Output (Raw Text and Markdown)

In [4]:
# Baseline output

from IPython.display import Markdown    # For pretty-printing Markdown output

messages = []

prompt = "Generate three different sample AWS CLI commands. Each command should be short."

add_user_message(messages, prompt)
response = chat(messages)

print(response)

Markdown(response)

# Sample AWS CLI Commands

Here are three short AWS CLI commands:

## 1. List S3 Buckets
```bash
aws s3 ls
```

## 2. Describe EC2 Instances
```bash
aws ec2 describe-instances
```

## 3. List IAM Users
```bash
aws iam list-users
```


# Sample AWS CLI Commands

Here are three short AWS CLI commands:

## 1. List S3 Buckets
```bash
aws s3 ls
```

## 2. Describe EC2 Instances
```bash
aws ec2 describe-instances
```

## 3. List IAM Users
```bash
aws iam list-users
```

### Solution

In [5]:
messages = []

prompt = "Generate three different sample AWS CLI commands. Each command should be short."

add_user_message(messages, prompt)
add_assistant_message(messages, "Here are all three commands without any comments:\n```bash")
response = chat(messages, stop_sequences=["```"])

print(response)


aws s3 ls s3://my-bucket/

aws ec2 describe-instances --instance-ids i-1234567890abcdef0

aws lambda invoke --function-name my-function output.json

